# Dublin House Prices by Post Code
## Maps
We'll use Pascal Bugnion's [excellent gmaps package](https://github.com/pbugnion/gmaps) to map our house data. First, we'll load our data and check it's ok.

In [1]:
import pandas as pd
import pickle
with open('../library/pickle/2016/prices_final_dublin_2016_pickle', 'r') as f:
    houses = pickle.load(f)
houses.head()

,Address,Date,Description,FullMarketPrice,Lat,Lon,Price,Size,VAT,eirKey,eirKeyName
0,"34 Mountpleasant Terrace, Dublin 6, D06 YC58, ...",2016-01-01,Second-Hand Dwelling house /Apartment,No,53.328587,-6.261495,170000.0,,No,D06,Rathmines
1,"2 Brighton Rd, Brighton Hall, Kerrymount, Dubl...",2016-01-04,Second-Hand Dwelling house /Apartment,No,53.258165,-6.174641,1150000.0,,No,D18,Stepaside
2,"24 Woodstown Meadow, Ballycullen, Dublin 16, D...",2016-01-04,Second-Hand Dwelling house /Apartment,No,53.273761,-6.327188,430000.0,,No,D16,Ballinteer
3,"28 Belton Park Gardens, Clontarf, Dublin 9, D0...",2016-01-04,Second-Hand Dwelling house /Apartment,No,53.375715,-6.226541,225000.0,,No,D09,Drumcondra
4,"48A Beaufield Park, Stillorgan, Dublin, A94 XH...",2016-01-04,Second-Hand Dwelling house /Apartment,No,53.290123,-6.203367,345000.0,,No,A94,Blackrock


Next, we'll create a `gmaps` object, `houseMap`, using an authentication key from [Google API](https://console.developers.google.com/apis/).

In [2]:
import gmaps
import json
with open('api.json', 'r') as f:
    keys = json.load(f)
gmaps.configure(api_key = keys['create_the_map'])

houseMap = gmaps.Map()

`houseMap` will show the locations of the houses sold in Dublin in 2016-to-date.

Having created our map object, we now  need to assemble some data for it. To do this, we'll create a copy of our Lat, Lon and Price columns in the `houses` data frame, make sure there are no null values, and convert each into a list.

As we only need out latitude and longitudinal details first, we create a list of lat-lon tuples and use this list to create a `Heatmap` layer which we add to the `houseMap` object. We call `houseMap` and voila.

In [3]:
forMapping = houses[['Lat', 'Lon', 'Price']].copy()
forMapping = forMapping.dropna(how='any')

lat = forMapping['Lat'].tolist()
lon = forMapping['Lon'].tolist()
price = forMapping['Price'].tolist()

count_data = []
[count_data.append((lat[i], lon[i])) for i in range(len(lat))]

count_layer = gmaps.Heatmap(data=count_data, point_radius = 15)
houseMap.add_layer(count_layer)

## House Sales in Dublin, 2016-to-Date

In [4]:
houseMap

There is a similar process to create the Prices map. This time, we called `WeightedHeapmap`, which takes a third, Price, value which it uses to color the heatmap.

In [5]:
price_mapping_data = []
[price_mapping_data.append((lat[i], lon[i], price[i])) for i in range(len(lat))]

price_layer = gmaps.WeightedHeatmap(data=price_mapping_data, point_radius=15)
priceMap = gmaps.Map()
priceMap.add_layer(price_layer)

# House Prices in Dublin, 2016-to-Date

In [6]:
priceMap